In [1]:
import os
os.chdir('./../../')

from src.basic_utils import *
from src.preprocess_utils import *

In [3]:
data_path = 'data/'

train = pd.read_json(data_path+'train.json')
val = pd.read_json(data_path+'val.json')
test = pd.read_json(data_path+'test.json')
# song_meta = load_json(data_path+'song_meta.json')
# genre_gn_all = pd.read_json(data_path+'genre_gn_all.json', typ = 'series')

##### train, val, test 데이터 축소

In [ ]:
all_song = Counter()
all_song = update_song(train, all_song)
all_song = update_song(val, all_song)
all_song = update_song(test, all_song)

In [ ]:
tmp_dict = pd.DataFrame.from_dict(all_song, orient = 'index')
print("before ", tmp_dict.shape)
tmp_dict = tmp_dict[tmp_dict[0] >= 3]
print("after ", tmp_dict.shape)

## 5회 이상 등장한 노래만 사용! ==> use_song에 list형태로 담음
use_song = list(tmp_dict['index'])

In [ ]:
## 삭제 작업
for i in tqdm(train.index):
    train.loc[i:i]['songs'].values[0] = list(set(train.loc[i:i]['songs'].values[0]) & set(use_song))
    train.loc[i:i]['tags'].values[0] = list(map(lambda x:x.lower(),train.loc[i:i]['tags'].values[0]))
    
## val, test도 동일하게 tag에서 대문자를 소문자로 바꿔준다. 
for i in tqdm(val.index):
    val.loc[i:i]['tags'].values[0] = list(map(lambda x:x.lower(),val.loc[i:i]['tags'].values[0]))
    
for i in tqdm(test.index):
    test.loc[i:i]['tags'].values[0] = list(map(lambda x:x.lower(),test.loc[i:i]['tags'].values[0]))    

In [ ]:
train.to_parquet('train(song_trim).parquet')
val.to_json('val(lower).json', orient='records')
test.to_json('test(lower).json', orient='records')

##### song tag dict 생성

In [ ]:
train_tmp = train[['tags', 'songs']]

song_tag_dict = make_c1_c2_dict(train_tmp, 'songs', 'tags')
write_json(song_tag_dict, data_path+'song_tag_dict.json')

tag_song_dict = make_c1_c2_dict(train_tmp, 'tags', 'songs')
write_json(tag_song_dict, data_path+'tag_song_dict.json')